# How to use multi-mapping and multiple results

Multi-mapping is a feature of Dapper that allows you to map a single row from a SQL query to multiple objects in C#. This is useful when you have a query that joins multiple tables and you want to split the result into separate entities. For example, you can map a row that contains information about an order, a customer, and a store to three different classes: SalesOrderHeader, Customer, and Store.

Multiple results is another feature of Dapper that allows you to execute multiple queries in a single SQL statement and map the results to multiple objects. This is useful when you want to select data from different tables or views with a single database query. For example, you can execute a stored procedure that returns two result sets: one for orders and one for customers, and map them to two collections of Order and Customer objects.

To use multi-mapping and multiple results with C# and Dapper, you need to use the <code>QueryMultiple</code> method, which returns a <code>GridReader</code> object that can read multiple result sets. Then, you can use the <code>Read&lt;T&gt;</code> method with a custom mapping function to map each result set to a collection of objects. The mapping function takes two or more parameters that represent the objects to be mapped from the row, and returns a new object that combines them. For example, you can use a <code>Tuple</code> or an anonymous type to return a pair of objects.

Here is an example of how to use multi-mapping and multiple results with C# and Dapper, using the SalesOrderHeader, Customer, and Store tables as an example. 

In [1]:
// Connection string
#load "AppSettings.cs"

In [2]:
#r "nuget:Microsoft.Data.SqlClient"
#r "nuget:System.ComponentModel"
#r "nuget:Dapper"

using Dapper;
using Microsoft.Data.SqlClient;
using System.Collections.Generic;
using System.Data;

Installed Packages Dapper, 2.1.28 Microsoft.Data.SqlClient, 5.1.2 System.ComponentModel, 4.3.0

In [3]:
public class SalesOrderHeader
{   
    public int SalesOrderID { get; set; }
    public int CustomerID { get; set; }   
    public float SubTotal { get; set; }    
}

public class Customer
{   
    public int CustomerID { get; set; } 
    public string AccountNumber { get; set; }
}

public class Store
{  
    public int BusinessEntityID { get; set; }
    public string Name { get; set; }   
}

In [4]:
using (var connection = new SqlConnection(connectionString))
{
    string query = @"
        SELECT o.*, c.* FROM Sales.SalesOrderHeader o
        INNER JOIN Sales.Customer c ON o.CustomerID = c.CustomerID
        WHERE o.OrderDate > '2011-05-31' AND o.CreditCardID = 19234;

        SELECT c.*, s.* FROM Sales.Customer c
        INNER JOIN Sales.Store s ON  s.BusinessEntityID= c.StoreID 
        WHERE c.StoreID = 300
    ";

    // Execute the query and get a GridReader object
    using (var gridReader = connection.QueryMultiple(query))
    {
        // Read the first result set and map each row to a SalesOrderHeader and a Customer object
        var ordersAndCustomers = gridReader.Read<SalesOrderHeader, Customer, Tuple<SalesOrderHeader, Customer>>(
            (order, customer) => Tuple.Create(order, customer),
            splitOn: "CustomerID"
        );

        // Read the second result set and map each row to a Customer and a Store object
        var customersAndStores = gridReader.Read<Customer, Store, Tuple<Customer, Store>>(
            (customer, store) => Tuple.Create(customer, store),
            splitOn: "StoreID"
        );

        // Do something with the results
        foreach (var pair in ordersAndCustomers)
        {
            Console.WriteLine($"Sales Order ID: {pair.Item1.SalesOrderID}, SubTotal: {pair.Item1.SubTotal}, Customer ID: {pair.Item2.CustomerID}, Account Number: {pair.Item2.AccountNumber}");
        }

        foreach (var pair in customersAndStores)
        {
            Console.WriteLine($"Customer ID: {pair.Item1.CustomerID}, Store ID: {pair.Item2.BusinessEntityID}, Name: {pair.Item2.Name}");
        }
    }
}

Sales Order ID: 56975, SubTotal: 14.98, Customer ID: 14363, Account Number: AW00014363
Sales Order ID: 68696, SubTotal: 38.88, Customer ID: 14363, Account Number: AW00014363
Sales Order ID: 70978, SubTotal: 138.48, Customer ID: 14363, Account Number: AW00014363
Customer ID: 573, Store ID: 300, Name: Nationwide Supply
Customer ID: 29488, Store ID: 300, Name: Nationwide Supply
